In [4]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.layers import Dense, Flatten, Dropout
from sklearn import preprocessing
import tensorflow as tf

In [5]:
Data = loadmat('./v5.mat')
testData = loadmat('./testing_v5.mat')

In [6]:
parm_mat = Data['parm_mat']
Ex = Data['Wav_mat']
testEx = testData['Wav_mat']

In [7]:
Input = [Ex[i][0] for i in range(Ex.shape[0] - 1)]
Input = np.array(Input)
Input.shape

(22320, 69, 129)

In [8]:
Output = parm_mat
n_sample = len(Input)
Output

array([[2.4 , 1.2 ],
       [2.4 , 1.19],
       [2.4 , 1.18],
       ...,
       [0.2 , 0.22],
       [0.2 , 0.21],
       [0.2 , 0.2 ]])

In [9]:
testInput = [testEx[i][0] for i in range(testEx.shape[0])]
testInput = np.array(testInput)

In [10]:
min_max_scaler = preprocessing.MinMaxScaler()  # normalization
Output_scale = min_max_scaler.fit_transform(Output)

In [11]:
index = np.arange(n_sample)
np.random.seed(1)
np.random.shuffle(index)

In [12]:
train_ratio = 0.7
x_train = Input[index[:int(0.7*n_sample)]]
x_test = Input[index[int(0.7*n_sample):]]
y_train = Output[index[:int(0.7*n_sample)]]
y_test = Output[index[int(0.7*n_sample):]]
x_train.shape

(15623, 69, 129)

In [13]:
img_x, img_y = 69, 129
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
print(x_train.shape)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)

(15623, 69, 129, 1)


In [21]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_x, img_y, 1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(2, activation='relu'))

In [22]:
# adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.009, epsilon=None, amsgrad=False)
model.compile(optimizer='Adam', loss='mse', metrics=['cosine_similarity'])

In [23]:
history = model.fit(x_train, y_train, validation_split=0.2, batch_size=50, epochs=90)

Epoch 1/90


2022-07-13 22:09:35.855864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


249/250 [============================>.] - ETA: 0s - loss: 0.0215 - cosine_similarity: 0.9895

2022-07-13 22:09:44.412949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 9s 36ms/step - loss: 0.0214 - cosine_similarity: 0.9895 - val_loss: 0.0013 - val_cosine_similarity: 0.9995
Epoch 2/90
250/250 [==============================] - 9s 36ms/step - loss: 9.4814e-04 - cosine_similarity: 0.9996 - val_loss: 5.1782e-04 - val_cosine_similarity: 0.9997
Epoch 3/90
250/250 [==============================] - 9s 36ms/step - loss: 4.8351e-04 - cosine_similarity: 0.9997 - val_loss: 2.5998e-04 - val_cosine_similarity: 0.9997
Epoch 4/90
250/250 [==============================] - 9s 36ms/step - loss: 3.1819e-04 - cosine_similarity: 0.9998 - val_loss: 2.0415e-04 - val_cosine_similarity: 0.9997
Epoch 5/90
250/250 [==============================] - 9s 36ms/step - loss: 2.8047e-04 - cosine_similarity: 0.9998 - val_loss: 1.7487e-04 - val_cosine_similarity: 0.9998
Epoch 6/90
250/250 [==============================] - 9s 36ms/step - loss: 2.6255e-04 - cosine_similarity: 0.9998 - val_loss: 2.3464e-04 - val_cosine_similarity: 0.9999
Epoch

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test)
score

In [ ]:
print(history.history.keys())

In [ ]:
loss = history.history['loss']
# val_loss = history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.title('Loss')
plt.xlabel('epoch')
plt.yscale('log')
plt.ylabel('loss')
plt.plot(epochs, loss, 'black', label='training loss')
# plt.plot(epochs, val_loss, 'red', label='validation loss')
plt.show()

In [ ]:
cos = history.history['cosine_similarity']
# val_acc = history.history['val_accuracy']
epochs = range(1, len(cos)+1)
plt.plot(epochs, cos, 'yellow', label='Training cos')
# plt.plot(epochs, val_acc, 'green', label='validation acc')
plt.xlabel('Epochs')
plt.ylabel('Cos')
plt.legend()
plt.show()

In [ ]:
output = model.predict(testInput)
output

In [231]:
np.min(val_acc)

0.7775407433509827

In [232]:
np.argmin(val_acc)

26